## Importing NIH Dataset (ZIP Format)

In [1]:
# Import relevant packages
import numpy as np
import os
from shutil import copyfile
from zipfile import ZipFile

# Download NIH dataset zip file
!wget -nc ftp://lhcftp.nlm.nih.gov/Open-Access-Datasets/Malaria/cell_images.zip

# Extract images if not already extracted
ROOT_DIR = os.path.join("/", "content")
if not os.path.isdir("cell_images"):
    print("Extracting images...")
    with ZipFile(os.path.join("cell_images.zip"), "r") as zipObj:
        zipObj.extractall()
    print("Done!")

--2020-03-01 19:05:23--  ftp://lhcftp.nlm.nih.gov/Open-Access-Datasets/Malaria/cell_images.zip
           => ‘cell_images.zip’
Resolving lhcftp.nlm.nih.gov (lhcftp.nlm.nih.gov)... 130.14.55.35, 2607:f220:41e:7055::35
Connecting to lhcftp.nlm.nih.gov (lhcftp.nlm.nih.gov)|130.14.55.35|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /Open-Access-Datasets/Malaria ... done.
==> SIZE cell_images.zip ... 353452851
==> PASV ... done.    ==> RETR cell_images.zip ... done.
Length: 353452851 (337M) (unauthoritative)

cell_images.zip     100%[===================>] 337.08M  74.5MB/s    in 4.7s    

2020-03-01 19:05:28 (71.2 MB/s) - ‘cell_images.zip’ saved [353452851]

Extracting images...
Done!


## Unzip Images, Resize, and Store in NumPy Arrays

In [2]:
# Install and import relevant packages
import numpy as np
import os
!pip install opencv-python
!apt update && apt install -y libsm6 libxext6 libxrender1
import cv2
from PIL import Image

# Create new folders to save rescaled images
if not os.path.isdir("RescaledSet"):
    os.mkdir("RescaledSet")
if not os.path.isdir("RescaledSet/Parasitized"):
    os.mkdir("RescaledSet/Parasitized")
if not os.path.isdir("RescaledSet/Uninfected"):
    os.mkdir("RescaledSet/Uninfected")

# Generate list of parasitized file names
ParasitizedFiles = os.listdir("cell_images/Parasitized/")
UninfectedFiles = os.listdir("cell_images/Uninfected/")

# Remove Thumb.db files
while 'Thumbs.db' in ParasitizedFiles: ParasitizedFiles.remove('Thumbs.db')   
while 'Thumbs.db' in UninfectedFiles: UninfectedFiles.remove('Thumbs.db')  

# Pre-allocate memory space for images
Parasitized = np.empty([13779,128,128,3])
Uninfected = np.empty([13779,128,128,3])

# Resize and load parasitized images
for i in range(13779):
    TempImage = cv2.imread('cell_images/Parasitized/'+ParasitizedFiles[i])
    ResizedImage = cv2.resize(TempImage, dsize=(128,128))
    Parasitized[i,:,:,:] = ResizedImage

# Resize and load uninfected images
for i in range(13779):
    TempImage = cv2.imread('cell_images/Uninfected/'+UninfectedFiles[i])
    ResizedImage = cv2.resize(TempImage, dsize=(128,128))
    Uninfected[i,:,:,:] = ResizedImage
    
print('Uninfected Dataset size is:',np.shape(Uninfected))
print('Parasitized Dataset size is:',np.shape(Parasitized))

     |████████████████████████████████| 28.2 MB 3.1 MB/s eta 0:00:01
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease          
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease                  
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done                
Building dependency tree       
Reading state information... Done
28 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree       
Reading state infor

## Generate Cross-Validation Indices for Training and Testing Sets

In [4]:
# Generate dataset labels
ParasitizedLabels = np.repeat([[0,1]], 13779, axis=0)
UninfectedLabels = np.repeat([[1,0]], 13779, axis=0)
Labels = np.concatenate((ParasitizedLabels,UninfectedLabels), axis=0)

# Generate image dataset
Dataset = np.concatenate((Parasitized, Uninfected), axis=0)

# Generate 5-fold cross-validation groups
CVIndices = np.random.permutation(Dataset.shape[0])
Index1, Index2, Index3, Index4, Index5 = CVIndices[:5512], CVIndices[5512:11024], CVIndices[11024:16536], CVIndices[16536:22048], CVIndices[22048:]

## Create List of Classification Layer Hyperparameters

In [6]:
# Import relevant neural network architecture packages 
from keras.applications.vgg16 import VGG16

## Train Model and Save Results as CSV Files

In [ ]:
# Import relevant packages for neural network training
import sys
import csv
if 'tensorflow' in sys.modules == False:
    %tensorflow_version 2.x
    import tensorflow as tf
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, BatchNormalization
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

!pip install scikit-learn
import sklearn
from sklearn.metrics import roc_curve
from sklearn.metrics import auc


# Create empty lists to store results
TrainLoss = []
TrainAcc = []
TestLoss = []
TestAcc = []
All_FPR = []
All_TPR = []
All_thresholds = []
All_AUC = []

for i in range(5):

    # Create the appropriate training and testing sets
    if i == 0:
        TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index3,:],Dataset[Index4,:]), axis=0)
        TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index2,:], Labels[Index3,:], Labels[Index4,:]), axis=0)
        TestImages = Dataset[Index5,:]
        TestLabels = Labels[Index5,:]
    elif i == 1:
        TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index3,:],Dataset[Index5,:]), axis=0)
        TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index2,:], Labels[Index3,:], Labels[Index5,:]), axis=0)
        TestImages = Dataset[Index4,:]
        TestLabels = Labels[Index4,:]
    elif i == 2:
        TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
        TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index2,:], Labels[Index4,:], Labels[Index5,:]), axis=0)
        TestImages = Dataset[Index3,:]
        TestLabels = Labels[Index3,:]
    elif i == 3:
        TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index3,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
        TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index3,:], Labels[Index4,:], Labels[Index5,:]), axis=0)
        TestImages = Dataset[Index2,:]
        TestLabels = Labels[Index2,:]
    else:
        TrainImages = np.concatenate((Dataset[Index2,:],Dataset[Index3,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
        TrainLabels = np.concatenate((Labels[Index2,:], Labels[Index3,:], Labels[Index4,:], Labels[Index5,:]), axis=0)
        TestImages = Dataset[Index1,:]
        TestLabels = Labels[Index1,:]

    base_model = VGG16(weights = "imagenet", include_top=False, input_shape = (128, 128, 3))

    # Store the fully connected layers
    Input = base_model.layers[0]
    ConvBlock1_1 = base_model.layers[1]
    ConvBlock1_2 = base_model.layers[2]
    ConvBlock1_Pool = base_model.layers[3]

    ConvBlock2_1 = base_model.layers[4]
    ConvBlock2_2 = base_model.layers[5]
    ConvBlock2_Pool = base_model.layers[6]
    ConvBlock3_1 = base_model.layers[7]
    ConvBlock3_2 = base_model.layers[8]
    ConvBlock3_3 = base_model.layers[9]
    ConvBlock3_Pool = base_model.layers[10]
    ConvBlock4_1 = base_model.layers[11]
    ConvBlock4_2 = base_model.layers[12]
    ConvBlock4_3 = base_model.layers[13]
    ConvBlock4_Pool = base_model.layers[14]
    ConvBlock5_1 = base_model.layers[15]
    ConvBlock5_2 = base_model.layers[16]
    ConvBlock5_3 = base_model.layers[17]
    ConvBlock5_Pool = base_model.layers[18]

    #NewLayer = ConvBlock5_3(ConvBlock5_2)


    # Reconstructing neural network architecture w/ batch normalization
    x = Input.output
    x = ConvBlock1_1(x)
    x = ConvBlock1_2(x)
    x = BatchNormalization()(x)
    x = ConvBlock1_Pool(x)
    x = ConvBlock2_1(x)
    x = ConvBlock2_2(x)
    x = BatchNormalization()(x)
    x = ConvBlock2_Pool(x)
    x = ConvBlock3_1(x)
    x = ConvBlock3_2(x)
    x = ConvBlock3_3(x)
    x = BatchNormalization()(x)
    x = ConvBlock3_Pool(x)
    x = ConvBlock4_1(x)
    x = ConvBlock4_2(x)
    x = ConvBlock4_3(x)
    x = BatchNormalization()(x)
    x = ConvBlock4_Pool(x)
    x = ConvBlock5_1(x)
    x = ConvBlock5_2(x)
    x = ConvBlock5_3(x)
    x = BatchNormalization()(x)
    x = ConvBlock5_Pool(x)

    ### INSERT REST OF LAYERS HERE ###
    x = Flatten()(x)
    x = Dense(1024, activation="relu")(x)
    x = BatchNormalization()(x) # INSERT BATCH NORMALIZATION LAYER HERE
    x = Dropout(0.5)(x)
    x = Dense(1024, activation="relu")(x)
    x = BatchNormalization()(x) # INSERT BATCH NORMALIZATION LAYER HERE
    x = Dropout(0.5)(x)
    predictions = Dense(2, activation="softmax")(x)
    model = Model(input = base_model.input, output = predictions)
    sgd = optimizers.SGD(learning_rate=1e-5, momentum=0.9, nesterov=True)
    model.compile(loss = "categorical_crossentropy", optimizer = sgd, metrics=["accuracy"])

    # Train model and evaluate performance
    print('We are now training cross-validation set #',i+1)
    Results = model.fit(TrainImages, TrainLabels, epochs=75, batch_size=64, validation_data=(TestImages,TestLabels), validation_freq=1)

    # Display and store performance results
    Results.history['loss'] = [round(l, 4) for l in Results.history['loss']]
    Results.history['accuracy'] = [round(l, 4) for l in Results.history['accuracy']]
    Results.history['val_loss'] = [round(l, 4) for l in Results.history['val_loss']]
    Results.history['val_accuracy'] = [round(l, 4) for l in Results.history['val_accuracy']]

    print('Training Loss:',Results.history['loss'])
    print('Training Accuracy:',Results.history['accuracy'])
    print('Validation Loss:',Results.history['val_loss'])
    print('Validation Accuracy:',Results.history['val_accuracy'])

    TrainLoss.append(Results.history['loss'])
    TrainAcc.append(Results.history['accuracy'])
    TestLoss.append(Results.history['val_loss'])
    TestAcc.append(Results.history['val_accuracy'])
    print('')


    # Predict values for test set
    Probabilities = model.predict(TestImages)

    # Calculate data for ROC curve
    FPR, TPR, thresholds = roc_curve(TestLabels[:,1], Probabilities[:,1])
    All_FPR.append(FPR)
    All_TPR.append(TPR)
    All_thresholds.append(thresholds)

# Save and export as CSV files
with open("BatchNorm_TrainLoss.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(TrainLoss)
with open("BatchNorm_TrainAcc.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(TrainAcc)
with open("BatchNorm_TestLoss.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(TestLoss)
with open("BatchNorm_TestAcc.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(TestAcc)
with open("BatchNorm_FPR.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(All_FPR)
with open("BatchNorm_TPR.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(All_TPR)
with open("BatchNorm_Thresholds.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(All_thresholds)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:123: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


We are now training cross-validation set # 1
Train on 22048 samples, validate on 5510 samples
Epoch 1/75
22048/22048 [==============================] - 55s 2ms/step - loss: 0.7193 - accuracy: 0.7221 - val_loss: 0.5677 - val_accuracy: 0.7074
Epoch 2/75
22048/22048 [==============================] - 53s 2ms/step - loss: 0.4274 - accuracy: 0.8382 - val_loss: 0.2298 - val_accuracy: 0.9042
Epoch 3/75
22048/22048 [==============================] - 53s 2ms/step - loss: 0.3346 - accuracy: 0.8795 - val_loss: 0.1804 - val_accuracy: 0.9318
Epoch 4/75
22048/22048 [==============================] - 53s 2ms/step - loss: 0.2775 - accuracy: 0.9012 - val_loss: 0.1638 - val_accuracy: 0.9410
Epoch 5/75
22048/22048 [==============================] - 53s 2ms/step - loss: 0.2417 - accuracy: 0.9141 - val_loss: 0.1545 - val_accuracy: 0.9445
Epoch 6/75
22048/22048 [==============================] - 53s 2ms/step - loss: 0.2272 - accuracy: 0.9217 - val_loss: 0.1541 - val_accuracy: 0.9465
Epoch 7/75
22048/22048 [

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:123: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


We are now training cross-validation set # 2
Train on 22046 samples, validate on 5512 samples
Epoch 1/75
22046/22046 [==============================] - 54s 2ms/step - loss: 0.7851 - accuracy: 0.6915 - val_loss: 0.3276 - val_accuracy: 0.8639
Epoch 2/75
22046/22046 [==============================] - 52s 2ms/step - loss: 0.4494 - accuracy: 0.8288 - val_loss: 0.2073 - val_accuracy: 0.9131
Epoch 3/75
22046/22046 [==============================] - 52s 2ms/step - loss: 0.3439 - accuracy: 0.8721 - val_loss: 0.1762 - val_accuracy: 0.9305
Epoch 4/75
22046/22046 [==============================] - 52s 2ms/step - loss: 0.2928 - accuracy: 0.8957 - val_loss: 0.1612 - val_accuracy: 0.9399
Epoch 5/75
22046/22046 [==============================] - 52s 2ms/step - loss: 0.2552 - accuracy: 0.9108 - val_loss: 0.1468 - val_accuracy: 0.9472
Epoch 6/75
22046/22046 [==============================] - 52s 2ms/step - loss: 0.2379 - accuracy: 0.9182 - val_loss: 0.1395 - val_accuracy: 0.9510
Epoch 7/75
22046/22046 [

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:123: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


We are now training cross-validation set # 3
Train on 22046 samples, validate on 5512 samples
Epoch 1/75
22046/22046 [==============================] - 54s 2ms/step - loss: 0.7612 - accuracy: 0.7068 - val_loss: 0.3721 - val_accuracy: 0.8380
Epoch 2/75
22046/22046 [==============================] - 52s 2ms/step - loss: 0.4484 - accuracy: 0.8320 - val_loss: 0.2241 - val_accuracy: 0.9100
Epoch 3/75
22046/22046 [==============================] - 53s 2ms/step - loss: 0.3502 - accuracy: 0.8697 - val_loss: 0.1856 - val_accuracy: 0.9292
Epoch 4/75
22046/22046 [==============================] - 52s 2ms/step - loss: 0.2864 - accuracy: 0.8952 - val_loss: 0.1686 - val_accuracy: 0.9376
Epoch 5/75
22046/22046 [==============================] - 52s 2ms/step - loss: 0.2545 - accuracy: 0.9107 - val_loss: 0.1558 - val_accuracy: 0.9427
Epoch 6/75
22046/22046 [==============================] - 52s 2ms/step - loss: 0.2279 - accuracy: 0.9199 - val_loss: 0.1489 - val_accuracy: 0.9461
Epoch 7/75
22046/22046 [

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:123: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


We are now training cross-validation set # 4
Train on 22046 samples, validate on 5512 samples
Epoch 1/75
22046/22046 [==============================] - 55s 2ms/step - loss: 0.7521 - accuracy: 0.7147 - val_loss: 0.4931 - val_accuracy: 0.7533
Epoch 2/75
22046/22046 [==============================] - 53s 2ms/step - loss: 0.4425 - accuracy: 0.8291 - val_loss: 0.2389 - val_accuracy: 0.9022
Epoch 3/75
22046/22046 [==============================] - 53s 2ms/step - loss: 0.3395 - accuracy: 0.8750 - val_loss: 0.2017 - val_accuracy: 0.9233
Epoch 4/75
22046/22046 [==============================] - 53s 2ms/step - loss: 0.2773 - accuracy: 0.8993 - val_loss: 0.1874 - val_accuracy: 0.9309
Epoch 5/75
22046/22046 [==============================] - 53s 2ms/step - loss: 0.2455 - accuracy: 0.9144 - val_loss: 0.1750 - val_accuracy: 0.9398
Epoch 6/75
22046/22046 [==============================] - 53s 2ms/step - loss: 0.2263 - accuracy: 0.9214 - val_loss: 0.1638 - val_accuracy: 0.9434
Epoch 7/75
22046/22046 [

### Without Batch Normalization

In [ ]:
# Import relevant packages for neural network training
import sys
import csv
if 'tensorflow' in sys.modules == False:
    %tensorflow_version 2.x
    import tensorflow as tf
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, BatchNormalization
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

!pip install scikit-learn
import sklearn
from sklearn.metrics import roc_curve
from sklearn.metrics import auc


# Create empty lists to store results
TrainLoss = []
TrainAcc = []
TestLoss = []
TestAcc = []
All_FPR = []
All_TPR = []
All_thresholds = []
All_AUC = []

for i in range(5):

    # Create the appropriate training and testing sets
    if i == 0:
        TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index3,:],Dataset[Index4,:]), axis=0)
        TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index2,:], Labels[Index3,:], Labels[Index4,:]), axis=0)
        TestImages = Dataset[Index5,:]
        TestLabels = Labels[Index5,:]
    elif i == 1:
        TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index3,:],Dataset[Index5,:]), axis=0)
        TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index2,:], Labels[Index3,:], Labels[Index5,:]), axis=0)
        TestImages = Dataset[Index4,:]
        TestLabels = Labels[Index4,:]
    elif i == 2:
        TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
        TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index2,:], Labels[Index4,:], Labels[Index5,:]), axis=0)
        TestImages = Dataset[Index3,:]
        TestLabels = Labels[Index3,:]
    elif i == 3:
        TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index3,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
        TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index3,:], Labels[Index4,:], Labels[Index5,:]), axis=0)
        TestImages = Dataset[Index2,:]
        TestLabels = Labels[Index2,:]
    else:
        TrainImages = np.concatenate((Dataset[Index2,:],Dataset[Index3,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
        TrainLabels = np.concatenate((Labels[Index2,:], Labels[Index3,:], Labels[Index4,:], Labels[Index5,:]), axis=0)
        TestImages = Dataset[Index1,:]
        TestLabels = Labels[Index1,:]

    base_model = VGG16(weights = "imagenet", include_top=False, input_shape = (128, 128, 3))

    # Reconstructing neural network architecture w/ batch normalization
    x = base_model.output
    x = Flatten()(x)
    x = Dense(1024, activation="relu")(x)
    x = BatchNormalization()(x) # INSERT BATCH NORMALIZATION LAYER HERE
    x = Dropout(0.5)(x)
    x = Dense(1024, activation="relu")(x)
    x = BatchNormalization()(x) # INSERT BATCH NORMALIZATION LAYER HERE
    x = Dropout(0.5)(x)
    predictions = Dense(2, activation="softmax")(x)
    model = Model(input = base_model.input, output = predictions)
    sgd = optimizers.SGD(learning_rate=1e-5, momentum=0.9, nesterov=True)
    model.compile(loss = "categorical_crossentropy", optimizer = sgd, metrics=["accuracy"])

    # Train model and evaluate performance
    print('We are now training cross-validation set #',i+1)
    Results = model.fit(TrainImages, TrainLabels, epochs=75, batch_size=64, validation_data=(TestImages,TestLabels), validation_freq=1)

    # Display and store performance results
    Results.history['loss'] = [round(l, 4) for l in Results.history['loss']]
    Results.history['accuracy'] = [round(l, 4) for l in Results.history['accuracy']]
    Results.history['val_loss'] = [round(l, 4) for l in Results.history['val_loss']]
    Results.history['val_accuracy'] = [round(l, 4) for l in Results.history['val_accuracy']]

    print('Training Loss:',Results.history['loss'])
    print('Training Accuracy:',Results.history['accuracy'])
    print('Validation Loss:',Results.history['val_loss'])
    print('Validation Accuracy:',Results.history['val_accuracy'])

    TrainLoss.append(Results.history['loss'])
    TrainAcc.append(Results.history['accuracy'])
    TestLoss.append(Results.history['val_loss'])
    TestAcc.append(Results.history['val_accuracy'])
    print('')


    # Predict values for test set
    Probabilities = model.predict(TestImages)

    # Calculate data for ROC curve
    FPR, TPR, thresholds = roc_curve(TestLabels[:,1], Probabilities[:,1])
    All_FPR.append(FPR)
    All_TPR.append(TPR)
    All_thresholds.append(thresholds)

# Save and export as CSV files
with open("NoBatchNorm_TrainLoss.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(TrainLoss)
with open("NoBatchNorm_TrainAcc.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(TrainAcc)
with open("NoBatchNorm_TestLoss.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(TestLoss)
with open("NoBatchNorm_TestAcc.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(TestAcc)
with open("NoBatchNorm_FPR.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(All_FPR)
with open("NoBatchNorm_TPR.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(All_TPR)
with open("NoBatchNorm_Thresholds.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(All_thresholds)

## Examine RAM Requirements

In [ ]:
# Examine RAM Usage
import sys
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)